In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from logging import Logger
from pathlib import Path
import gc
# noinspection PyUnresolvedReferences
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
PROJECT_DIR = Path().resolve().parents[0]
os.chdir(PROJECT_DIR)
SEED: Final = 42

In [2]:
from utils.setup import load_param, easy_logger, get_device
from utils.torch import load_model, torch_fix_seed
from torch.utils.tensorboard.writer import SummaryWriter
from ignite.handlers import Checkpoint
from run_for_KGC import main_function
logger = easy_logger('log/test.log')

In [3]:
args = load_param('saved_models/1229/01/param.pkl')
device = get_device(device_name='cpu', logger=logger)
# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.no_use_pe = False
args.max_len=512
args.tensorboard_dir=None
args.only_load_trainer_evaluator=True
args.pre_train=False
args.use_for_challenge090 = False
del args.optuna_file, args.device_name, args.pid
del args.study_name, args.n_trials
logger.info(args)

2022-12-30 21:19:47 - INFO - 991045332.py - 15 - Namespace(notebook=False, train_anyway=False, logfile='saved_models/1229/01/log.log', param_file='saved_models/1229/01/param.pkl', tensorboard_dir=None, checkpoint_dir='saved_models/1229/01/checkpoint/', model_path='saved_models/1229/01/model.pth', model_version='03', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, console_level='info', pre_train=False, train_valid_test=True, only_train=False, use_for_challenge100=False, use_for_challenge075=False, use_title=None, do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_token_s=0, cls_token_s=1, mask_token_s=2, sep_token_s=3, bos_token_s=4, embedding_dim=128, entity_embedding_dim=128, relation_embedding_dim=64, separate_he

In [ ]:
torch_fix_seed(seed=SEED)
model, data_dict = main_function(args, logger=logger)
encoder_last_layer = model.transformer.layers[-1]
model.eval()
dataset_train, dataset_valid, dataset_test = data_dict['datasets']
triple = dataset_train.triple
data_helper = data_dict['data_helper']
evaluator = data_dict['train_items']['evaluator']
load_model(model, args.model_path, device)
print(len(data_helper.processed_train_triple), len(triple))

In [5]:
entities = data_helper.processed_entities
d_e = {e: i for i, e in enumerate(entities)}
relations = data_helper.processed_relations
d_r = {r: i for i, r in enumerate(relations)}
triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=['head', 'relation', 'tail']) 
# story_entities = triple_df
# triple_df

In [6]:
def extract(model, target, inputs):
    feature = None

    def forward_hook(module, inputs, _):
        # 順伝搬の出力を features というグローバル変数に記録する
        global features
        # 1. detach でグラフから切り離す。
        # 2. clone() でテンソルを複製する。モデルのレイヤーで ReLU(inplace=True) のように
        #    inplace で行う層があると、値がその後のレイヤーで書き換えられてまい、
        #    指定した層の出力が取得できない可能性があるため、clone() が必要。
        x, _, _ = inputs
        outputs = module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    # コールバック関数を登録する。
    handle = target.register_forward_hook(forward_hook)

    # 推論する
    model.eval()
    model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    # コールバック関数を解除する。
    handle.remove()

    return features

In [7]:
def heatmap_(index):
    input_ = dataset[index]
    # display([(i, entities[h], relations[r], entities[t]) for i, (h, r, t) in enumerate(input_)])
    features = extract(model, encoder_last_layer.self_attn, dataset[0][None, :])[0]
    df_atten = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_)])
    df_atten.columns=['head', 'relation', 'tail',] + [f'atten_from{i}' for i in range(len(df_atten.columns)-3)]
    display(df_atten)
    # ヒートマップの作成
    # sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
    for i in range(len(df_atten)):
        display(i, df_atten.iloc[i,:3].tolist())
        display(df_atten.sort_values(f'atten_from{i}', ascending=False).iloc[:10,:3])
        print("----------")
    plt.show()
# heatmap_(300)

In [8]:
def make_heatmap2_(input_):
    assert len(input_) == 1
    features = extract(model, encoder_last_layer.self_attn, input_)[0]
    df_atten = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_atten.columns=['head', 'relation', 'tail',] + [f'atten_from{i}' for i in range(len(df_atten.columns)-3)]
    return df_atten

In [25]:
story_entities = triple_df['head'].tolist()
print(len(story_entities))

14546


In [10]:
def make_ranking(_from, _to, _predicate, _whom, _subject, _why, _what, _where):
    if not (_from is None and _to is None):
        _start_index = story_entities.index(_from)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(_to)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            [d_e['<bos_e>'],d_r['<bos_r>'],d_e['<bos_e>']],
            [d_e['<mask_e>'],d_r['kgc:hasPredicate'],d_e[_predicate]],
            [d_e['<mask_e>'],d_r['kgc:whom'],d_e[_whom]],
            [d_e['<mask_e>'],d_r['kgc:subject'],d_e[_subject]],
            [d_e['<mask_e>'],d_r['kgc:why'],d_e[_why]],
            [d_e['<mask_e>'],d_r['kgc:what'],d_e[_what]],
            [d_e['<mask_e>'],d_r['kgc:where'],d_e[_where]],
        ]
    )
    mask_ = torch.tensor(
        [
            [False, False, False],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
            [True, False, True],
        ]
    )
    questions = torch.cat([triple[_start_index: _end_index], question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros(_end_index-_start_index,3), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)
    tmp = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred.to('cpu'), dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            predicate_, whom_, subj_, why_, what_, where_ = ans_
            tmp.append([entities[predicate_], entities[whom_], entities[subj_], entities[why_], entities[what_], entities[where_]])
    df_ranking = pd.DataFrame(tmp, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = make_heatmap2_(questions)
    return df_ranking, df_attension

In [11]:
MASK_E = '<mask_e>'
KILL = 'word.predicate:kill'
TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

### まだらの紐
Who killed Julia? (criminal & explanation)
被害者: Julia
犯人: Roylott
犯行に用いたもの: snake
犯行動機: 母の相続財産を独占したい

In [ ]:
victim = 'SpeckledBand:Julia'
killer = 'SpeckledBand:Roylott'
df_ranking_SpeckledBand, df_attension_SpeckledBand = make_ranking(
    'SpeckledBand:340', 'SpeckledBand:400', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_SpeckledBand.index[df_ranking_SpeckledBand['subject']==killer].tolist())
display(df_ranking_SpeckledBand.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SpeckledBand)
for i in range(len_):
    display(i, df_attension_SpeckledBand.iloc[i,:3].tolist())
    display(df_attension_SpeckledBand.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

In [ ]:
victim = 'SpeckledBand:Julia'
df_ranking_SpeckledBand, df_attension_SpeckledBand = make_ranking(
    'SpeckledBand:330', 'SpeckledBand:400', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_SpeckledBand.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SpeckledBand)
for i in range(len_-20, len_):
    display(i, df_attension_SpeckledBand.iloc[i,:3].tolist())
    display(df_attension_SpeckledBand.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 悪魔の足跡１
Who killed the victims? (criminal & explanation)
被害者: Brenda
犯人: Mortimer
犯行動機: 財産

In [ ]:
victim = 'DevilsFoot:Brenda'
killer = 'DevilsFoot:Mortimer'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_DevilsFoot1.index[df_ranking_DevilsFoot1['subject'] == killer].tolist()[0])
display(df_ranking_DevilsFoot1.iloc[:20,:])

# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

In [18]:
victim = 'DevilsFoot:Brenda'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    'DevilsFoot:400', 'DevilsFoot:488', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_DevilsFoot1.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_-20, len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:notExist,DevilsFoot:Mortimer,DevilsFoot:Sterndale,ACaseOfIdentity:Windibank,ACaseOfIdentity:Windibank,DevilsFoot:Sterndale
1,<bos_e>,DevilsFoot:Sterndale,DevilsFoot:Mortimer,DevilsFoot:Case_1,AllTitle:Watson,DancingMen:Cubitt
2,word.predicate:open,AllTitle:Holmes,ACaseOfIdentity:Windibank,SpeckledBand:Roylott,DancingMen:Cubitt,ACaseOfIdentity:Windibank
3,word.predicate:say,AllTitle:Watson,AllTitle:Holmes,DevilsFoot:Sterndale,DevilsFoot:Sterndale,word.predicate:exist
4,word.predicate:exist,ACaseOfIdentity:Windibank,DancingMen:Cubitt,DevilsFoot:Brenda,SpeckledBand:Roylott,SpeckledBand:Roylott
5,word.predicate:go,DevilsFoot:Brenda,DevilsFoot:Brenda,DevilsFoot:Mortimer,DateTime:1883-04-02T04:00:00,word.predicate:examine
6,DevilsFoot:Sterndale,ResidentPatient:Blessington,<bos_e>,word.predicate:notExist,DevilsFoot:Mortimer,SilverBlaze:police
7,word.predicate:have,DancingMen:Cubitt,AbbeyGrange:Jack_Croker,ResidentPatient:Blessington,DevilsFoot:Paper_package,CrookedMan:Nancy
8,AllTitle:Watson,word.predicate:examine,ResidentPatient:Blessington,SilverBlaze:police,word.predicate:wear,AllTitle:Watson
9,word.predicate:notWant,SpeckledBand:Roylott,DevilsFoot:Roundhay,word.predicate:wear,CrookedMan:Henry,word.predicate:notExist


385

['DevilsFoot:485', 'kgc:what', 'DevilsFoot:gun']

,head,relation,tail,atten_from385
159,DevilsFoot:435,kgc:subject,DevilsFoot:powder_medicine,0.014904
392,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.013918
386,DevilsFoot:485,kgc:infoSource,DevilsFoot:Sterndale,0.011751
389,DevilsFoot:486,kgc:infoSource,DevilsFoot:Sterndale,0.011196
163,DevilsFoot:436,kgc:subject,DevilsFoot:Sterndale,0.010913
374,DevilsFoot:482,kgc:what,DevilsFoot:fire,0.010886
167,DevilsFoot:436,kgc:infoSource,DevilsFoot:Sterndale,0.010255
388,DevilsFoot:486,kgc:subject,DevilsFoot:Mortimer,0.009985
344,DevilsFoot:474,kgc:infoSource,DevilsFoot:Sterndale,0.009746
384,DevilsFoot:485,kgc:subject,DevilsFoot:Sterndale,0.009526


----------


...

----------


404

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from404
159,DevilsFoot:435,kgc:subject,DevilsFoot:powder_medicine,0.021879
392,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.014640
374,DevilsFoot:482,kgc:what,DevilsFoot:fire,0.010904
393,DevilsFoot:487,kgc:hasProperty,word.predicate:gravel,0.010760
372,DevilsFoot:482,kgc:subject,DevilsFoot:Sterndale,0.009997
163,DevilsFoot:436,kgc:subject,DevilsFoot:Sterndale,0.009357
167,DevilsFoot:436,kgc:infoSource,DevilsFoot:Sterndale,0.009331
157,DevilsFoot:434,kgc:infoSource,DevilsFoot:Sterndale,0.009146
379,DevilsFoot:483,kgc:infoSource,DevilsFoot:Sterndale,0.009112
376,DevilsFoot:482,kgc:infoSource,DevilsFoot:Sterndale,0.008720


----------


### 悪魔の足跡2
Who killed the victims? (criminal & explanation)
被害者: Mortimer
犯人: 
犯行動機: 恋人の敵

In [ ]:
victim = 'DevilsFoot:Mortimer'
killer = 'DevilsFoot:Sterndale'
df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_DevilsFoot1.index[df_ranking_DevilsFoot1['subject'] == killer].tolist()[0])
display(df_ranking_DevilsFoot1.iloc[:20,:])

# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot1)
for i in range(len_):
    display(i, df_attension_DevilsFoot1.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot1.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

In [17]:
victim = 'DevilsFoot:Mortimer'
df_ranking_DevilsFoot2, df_attension_DevilsFoot2 = make_ranking(
    'DevilsFoot:400', 'DevilsFoot:488', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_DevilsFoot2.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_DevilsFoot2)
for i in range(len_-20, len_):
    display(i, df_attension_DevilsFoot2.iloc[i,:3].tolist())
    display(df_attension_DevilsFoot2.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:notExist,DevilsFoot:Mortimer,DevilsFoot:Sterndale,ACaseOfIdentity:Windibank,ACaseOfIdentity:Windibank,DevilsFoot:Sterndale
1,<bos_e>,DevilsFoot:Sterndale,DevilsFoot:Mortimer,DevilsFoot:Sterndale,AllTitle:Watson,DancingMen:Cubitt
2,word.predicate:say,ACaseOfIdentity:Windibank,ACaseOfIdentity:Windibank,DevilsFoot:Brenda,DevilsFoot:Sterndale,ACaseOfIdentity:Windibank
3,word.predicate:open,DevilsFoot:Brenda,AllTitle:Holmes,SpeckledBand:Roylott,DancingMen:Cubitt,SpeckledBand:Roylott
4,word.predicate:go,AllTitle:Holmes,<bos_e>,DevilsFoot:Case_1,DevilsFoot:Mortimer,word.predicate:exist
5,word.predicate:exist,ResidentPatient:Blessington,AbbeyGrange:Jack_Croker,word.predicate:notExist,DateTime:1883-04-02T04:00:00,word.predicate:examine
6,DevilsFoot:Sterndale,DevilsFoot:Roundhay,DevilsFoot:Brenda,ResidentPatient:Blessington,SpeckledBand:Roylott,SilverBlaze:police
7,word.predicate:notWant,AllTitle:Watson,DevilsFoot:Roundhay,DevilsFoot:window,word.predicate:wear,CrookedMan:Nancy
8,word.predicate:have,SpeckledBand:Roylott,DancingMen:Cubitt,DevilsFoot:Mortimer,DevilsFoot:Paper_package,AllTitle:Watson
9,word.predicate:come,DancingMen:Cubitt,CrookedMan:Nancy,SilverBlaze:police,DevilsFoot:Porter,DevilsFoot:Mortimer


385

['DevilsFoot:485', 'kgc:what', 'DevilsFoot:gun']

,head,relation,tail,atten_from385
159,DevilsFoot:435,kgc:subject,DevilsFoot:powder_medicine,0.014658
392,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.013619
386,DevilsFoot:485,kgc:infoSource,DevilsFoot:Sterndale,0.011740
389,DevilsFoot:486,kgc:infoSource,DevilsFoot:Sterndale,0.011217
163,DevilsFoot:436,kgc:subject,DevilsFoot:Sterndale,0.010927
374,DevilsFoot:482,kgc:what,DevilsFoot:fire,0.010693
167,DevilsFoot:436,kgc:infoSource,DevilsFoot:Sterndale,0.010366
388,DevilsFoot:486,kgc:subject,DevilsFoot:Mortimer,0.009774
344,DevilsFoot:474,kgc:infoSource,DevilsFoot:Sterndale,0.009723
384,DevilsFoot:485,kgc:subject,DevilsFoot:Sterndale,0.009457


----------


...

----------


404

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from404
159,DevilsFoot:435,kgc:subject,DevilsFoot:powder_medicine,0.022143
392,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.013943
393,DevilsFoot:487,kgc:hasProperty,word.predicate:gravel,0.010018
372,DevilsFoot:482,kgc:subject,DevilsFoot:Sterndale,0.009976
374,DevilsFoot:482,kgc:what,DevilsFoot:fire,0.009647
167,DevilsFoot:436,kgc:infoSource,DevilsFoot:Sterndale,0.009636
163,DevilsFoot:436,kgc:subject,DevilsFoot:Sterndale,0.009596
157,DevilsFoot:434,kgc:infoSource,DevilsFoot:Sterndale,0.009151
379,DevilsFoot:483,kgc:infoSource,DevilsFoot:Sterndale,0.009140
376,DevilsFoot:482,kgc:infoSource,DevilsFoot:Sterndale,0.008515


----------


### 背中の曲がった男: 
Why did Barclay die?
被害者: Barclay
犯人: 
犯行動機: 

In [ ]:
victim = 'CrookedMan:Barclay'
killer = 'CrookedMan:apoplexia'
df_ranking_CrookedMan, df_attension_CrookedMan = make_ranking(
    None, None, DIE, MASK_E, victim, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_CrookedMan.index[df_ranking_CrookedMan['why'] == killer].tolist()[0])
display(df_ranking_CrookedMan.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_CrookedMan)
for i in range(len_-20, len_):
    display(i, df_attension_CrookedMan.iloc[i,:3].tolist())
    display(df_attension_CrookedMan.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

In [ ]:
victim = 'CrookedMan:Barclay'
df_ranking_CrookedMan, df_attension_CrookedMan = make_ranking(
    'CrookedMan:300', 'CrookedMan:372', DIE, MASK_E, victim, MASK_E, MASK_E, MASK_E)

display(df_ranking_CrookedMan.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_CrookedMan)
for i in range(len_-20, len_):
    display(i, df_attension_CrookedMan.iloc[i,:3].tolist())
    display(df_attension_CrookedMan.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 僧坊荘園
Who killed Lord Blackenstall? (criminal & explanation)
被害者: Sir_Eustace_Brackenstall
犯人: 
犯行動機: 

In [ ]:
victim = 'AbbeyGrange:Sir_Eustace_Brackenstall'
killer = 'AbbeyGrange:Jack_Croker'
df_ranking_AbbeyGrange, df_attension_AbbeyGrange = make_ranking(
    None, None, KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

print(f"{killer}, ranking", df_ranking_AbbeyGrange.index[df_ranking_AbbeyGrange['subject'] == killer].tolist()[0])
display(df_ranking_AbbeyGrange.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_AbbeyGrange)
for i in range(len_):
    display(i, df_attension_AbbeyGrange.iloc[i,:3].tolist())
    display(df_attension_AbbeyGrange.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

In [ ]:
victim = 'AbbeyGrange:Sir_Eustace_Brackenstall'
df_ranking_AbbeyGrange, df_attension_AbbeyGrange = make_ranking(
    'AbbeyGrange:350', 'AbbeyGrange:413', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_AbbeyGrange.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_AbbeyGrange)
for i in range(len_-20, len_):
    display(i, df_attension_AbbeyGrange.iloc[i,:3].tolist())
    display(df_attension_AbbeyGrange.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 入院患者
Who killed Blessington? (criminal & explanation)
被害者: Blessington
犯人: 3人
犯行動機: 

In [ ]:
victim = 'ResidentPatient:Blessington'
df_ranking_Blessington, df_attension_Blessington = make_ranking(
    'ResidentPatient:250', 'ResidentPatient:323', KILL, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_Blessington.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_Blessington)
for i in range(len_-20, len_):
    display(i, df_attension_Blessington.iloc[i,:3].tolist())
    display(df_attension_Blessington.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 白銀
Who took out the White Silver Blaze? (criminal & explanation) 
被害者: Silver_Blaze
犯人: 
犯行動機: 

In [ ]:
victim = 'SilverBlaze:Silver_Blaze'
df_ranking_SilverBlaze, df_attension_SilverBlaze = make_ranking(
    'SilverBlaze:330', 'SilverBlaze:396', BRING, MASK_E, MASK_E, MASK_E, victim, MASK_E)

display(df_ranking_SilverBlaze.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SilverBlaze)
for i in range(len_-20, len_):
    display(i, df_attension_SilverBlaze.iloc[i,:3].tolist())
    display(df_attension_SilverBlaze.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 花嫁失踪事件（同一事件）
hozmaの失踪の名瀬を探る
被害者: ACaseOfIdentity:Hozma
犯人: 
犯行動機: 

In [ ]:
victim = 'ACaseOfIdentity:Hosmer'
df_ranking_ACaseOfIdentity, df_attension_ACaseOfIdentity = make_ranking(
    'ACaseOfIdentity:510', 'ACaseOfIdentity:578', HIDE, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_ACaseOfIdentity.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_ACaseOfIdentity)
for i in range(len_-20, len_):
    display(i, df_attension_ACaseOfIdentity.iloc[i,:3].tolist())
    display(df_attension_ACaseOfIdentity.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")
